In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import pandas as pd
import numpy as np

connect to db

In [2]:
sparql = SPARQLWrapper('https://dati.camera.it/sparql')

Create dataframe of the deputati from a legislation

In [3]:
dep = pd.read_csv('18esima_Leg/dep_18.csv')[['persona', 'nomeGruppo', 'nome', 'cognome']]
a = dep['cognome']+ ' ' + dep['nome']
dep['Parlamentare'] = a
dep.drop(['nome', 'cognome'], axis = 1, inplace= True)

a = []
for i in range(dep.shape[0]):
    a.append(dep['persona'][i][39:])

dep['id'] = a
dep.drop('persona', inplace= True, axis = 1)

p = []
dat = []
for i in dep['nomeGruppo']:
    e = i.split()
    p.append(e[-2][1:-1])
    d = e[-1].replace(')', '')
    dat.append(datetime.strptime(d[-10:], '%d.%m.%Y'))

dep['Partito'] = p
dep['dataIngresso'] = dat
dep.drop('nomeGruppo', axis = 1, inplace= True)

first_df = pd.DataFrame(columns= dep.columns)

for i in np.unique(dep['id']):
    if len(dep[dep['id'] == i]) < 2:
        first_df = pd.concat([first_df, dep[dep['id'] == i]])
    else:
        duplicate = dep[dep['id'] == i]
        first_df = pd.concat([first_df, duplicate.head(1)])


In [8]:
sparql.setReturnFormat(JSON)

leg = 18
year = '2021'

# deputati of a specific legislation
for m in range(1,13):
   final_df = first_df.copy()
   month = f'{m:02d}'
   for d in range(1,32):
      day = f'{d:02d}'
      sparql.setQuery("""
      select distinct ?votazione ?data ?titolo ?numeroVotazione ?cognome ?nome ?espressione ?infoAssenza ?deputato where {
      ?votazione a ocd:votazione;
         dc:date '"""+year+month+day+"""';
         dc:date ?data;
         rdfs:label ?titolo;
         dc:description ?descrizione;
         dc:identifier ?numeroVotazione.


      ## voti espressi
      ?voto a ocd:voto;
         ocd:rif_votazione ?votazione;
         dc:type ?espressione;
         ocd:rif_deputato ?deputato.
      OPTIONAL{?voto dc:description ?infoAssenza}


      ## info deputato
      ?deputato foaf:surname ?cognome; foaf:firstName ?nome  


      } ORDER BY ?cognome
      
      """
      )
      try:
         ret = sparql.queryAndConvert()
         res = ret["results"]["bindings"]
      except Exception as e:
         print(e)

      dep = []
      vot = []
      numero = []
      for i in res:
         dep.append((i.get('deputato')).get('value')[40:46])
         vot.append((i.get('espressione')).get('value'))
         numero.append((i.get('numeroVotazione')).get('value'))

      votazione = pd.DataFrame([dep,vot, numero], index = ['id', 'Voto1', 'numero']).T
      
      for i in list(set(votazione['numero'])):
         espressione = votazione[votazione['numero'] == i][['id', 'Voto1']]
         espressione.columns= ('id', i)
         final_df = pd.merge(final_df, espressione, how = 'left')
   
      
   final_df.to_csv('xviii-'+year+'-'+month+'.csv', index=False)


# se numero NA di un parlamentare è maggiore 

In [9]:
dic_21 = pd.read_csv('xviii-2021-12.csv')

In [10]:
dizionario_espressioni = {'Assente': '0', 'Favorevole': '1', 'Non ha votato': '0', 'Contrario': '-1', 'Astensione': '2', 'In missione': '0'}
for i in dic_21.columns[4:]:
    dic_21[i] = dic_21[i].map(dizionario_espressioni)

dic_21.fillna('0', inplace= True)
    

In [11]:
dic_21

,Parlamentare,id,Partito,dataIngresso,607017,607005,607022,607023,607020,607014,...,607007,608008,608001,608005,608009,608004,608002,608006,608007,608003
0,BASINI GIUSEPPE,200049,LEGA,2018-03-27,-1,-1,1,1,-1,-1,...,-1,0,0,0,0,0,0,0,0,0
1,BERSANI PIER LUIGI,300026,MISTO,2018-03-23,-1,-1,0,1,-1,-1,...,-1,1,1,1,2,1,1,1,1,1
2,LETTA ENRICO,300127,PD,2018-03-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MINNITI DOMENICO,300146,PD,2018-03-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,FRANCESCHINI DARIO,300246,PD,2018-03-27,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,GIORGETTI GIANCARLO,50115,LEGA,2018-03-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
656,PAROLO UGO,50123,LEGA,2018-03-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
657,FOTI TOMMASO,50204,FDI,2018-03-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
658,RUSSO PAOLO,50213,FI,2018-03-27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
